In [2]:
import pandas as pd 
import yfinance as yf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

In [3]:
def clean_data(data):
    clean = data[['Date','Tweet content']]
    
    return clean


def get_sentiment_score(df):
    for index,row in df.iterrows():
        tweet = row['Tweet content']
        df.at[index,'sentiment'] = sia.polarity_scores(tweet)['compound']
    
    temp = df[df['sentiment'] != 0.0]
    
    return temp


def get_daily_sentiment(data):
    return data.groupby('Date').mean()

def reset_index(data):
    data['short'] = 0
    data['return'] = 0
    data['date'] = data.index
    data.index = range(0,len(data))
    
    return data

def get_stock_prices(ticker,sentiment_score):
    return yf.download(ticker,sentiment_score['date'][0],sentiment_score['date'][len(sentiment_score)-1])

def match_open_close(sentiment_data, stock_prices):
    sentiment_data['Open'] = 0.0
    sentiment_data['Close'] = 0.0
    
    for j in range(0,len(sentiment_data)-1):
        for i in range(0,len(stock_prices)-1):
            if(sentiment_data.date[j] == str(stock_prices.index[i].date())):
                sentiment_data['Open'][j] = stock_prices['Open'][i]
                sentiment_data['Close'][j] = stock_prices['Close'][i]
                break
                
    return sentiment_data


def short(data):
    for index,row in data.iterrows():
        if(row['sentiment'] < 0):
            cnt = 1
            while(data.at[index+1,'Open'] != 0.0 and cnt <= 2):
                data.at[index+cnt,'short'] = 1
                cnt+=1
                
    return data

def calculate_return(data):
    for index,row in data.iterrows():
        if(row['short'] == 1):
            data.at[index,'return'] = 100*(row['Open'] - row['Close'])
            
    gain = 0
    for index,row in data.iterrows():
        if(row['return'] != 0):
            gain = gain+int(row['return'])
            
    print('Total Return: ', gain)
    return data

In [4]:
amzn_tweets = clean_data(pd.read_excel('tweet.xlsx','Stream'))
amzn_senti = get_sentiment_score(amzn_tweets)
amzn_senti = get_daily_sentiment(amzn_senti)
amzn_senti = reset_index(amzn_senti)
amzn = get_stock_prices('AMZN',amzn_senti)
amzn_senti = match_open_close(amzn_senti,amzn)
amzn_return = calculate_return(short(amzn_senti))

[*********************100%***********************]  1 of 1 downloaded


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Total Return:  491


In [5]:
amzn_return[amzn_return['short'] == 1]

,sentiment,short,return,date,Open,Close
25,-0.178977,1,-959,2016-04-25,616.61,626.20
26,0.130744,1,928,2016-04-26,626.17,616.88
27,0.100881,1,522,2016-04-27,611.80,606.57


In [9]:
aapl_tweets = clean_data(pd.read_excel('aapl_tweets.xlsx','Stream'))
aapl_senti = get_sentiment_score(aapl_tweets)
aapl_senti = get_daily_sentiment(aapl_senti)
aapl_senti = reset_index(aapl_senti)
aapl = get_stock_prices('AAPL',aapl_senti)
aapl_senti = match_open_close(aapl_senti,aapl)
aapl_return = calculate_return(short(aapl_senti))

[*********************100%***********************]  1 of 1 downloaded


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Total Return:  141


In [11]:
aapl_return[aapl_return['short']==1]

,sentiment,short,return,date,Open,Close
33,0.058316,1,76,2016-05-05,94.00,93.24
34,0.043765,1,65,2016-05-06,93.37,92.72


In [10]:
googl_tweets = clean_data(pd.read_excel('googl.xlsx','Stream'))
googl_senti = get_sentiment_score(googl_tweets)
googl_senti = get_daily_sentiment(googl_senti)
googl_senti = reset_index(googl_senti)
googl = get_stock_prices('GOOGL',googl_senti)
googl_senti = match_open_close(googl_senti,googl)
googl_return = calculate_return(short(googl_senti))

[*********************100%***********************]  1 of 1 downloaded


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Total Return:  1061


In [6]:
fb_tweets = clean_data(pd.read_excel('fb_tweets.xlsx','Stream'))
fb_senti = get_sentiment_score(fb_tweets)
fb_senti = get_daily_sentiment(fb_senti)
fb_senti = reset_index(fb_senti)
fb = get_stock_prices('FB',fb_senti)
fb_senti = match_open_close(fb_senti,fb)
fb_return = calculate_return(short(fb_senti))

[*********************100%***********************]  1 of 1 downloaded


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Total Return:  2


In [8]:
fb_return[fb_return['short'] == 1]

,sentiment,short,return,date,Open,Close
39,0.153934,1,-87,2016-05-10,119.62,120.50
40,0.186217,1,89,2016-05-11,120.41,119.52


In [21]:
yf.download('GOOGL','2017-09-2','2019-02-1')

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-09-01,957.47,958.33,950.28,951.99,951.99,1042900
2017-09-05,946.86,951.39,935.60,941.48,941.48,1457400
2017-09-06,943.87,944.50,932.68,942.02,942.02,1391500
2017-09-07,944.25,950.50,937.53,949.89,949.89,1116600
2017-09-08,949.70,950.70,940.01,941.41,941.41,999900
2017-09-11,947.20,952.68,941.00,943.29,943.29,1337400
2017-09-12,946.92,948.09,937.50,946.65,946.65,1284800
2017-09-13,945.50,952.85,944.74,950.44,950.44,1095400
2017-09-14,946.00,948.03,938.36,940.13,940.13,1427400


In [12]:
sia.polarity_scores("Algoritmic trading is amazing")['compound']

0.5859

In [16]:
sia.polarity_scores("Algoritmic trading is amazing!")['compound']

0.6239

In [17]:
sia.polarity_scores("Algoritmic trading is ok")['compound']

0.296

In [22]:
sia.polarity_scores("Algoritmic trading is so bad")['compound']

-0.6696

In [21]:
sia.polarity_scores("Algoritmic trading is SO bad")['compound']

-0.6732

In [42]:
googl = pd.read_excel('googl.xlsx','Stream')
print('Length: ', len(googl))

Length:  29385


In [30]:
googl.columns

Index(['Tweet Id', 'Date', 'Hour', 'User Name', 'Nickname', 'Bio',
       'Tweet content', 'Favs', 'RTs', 'Latitude', 'Longitude', 'Country',
       'Place (as appears on Bio)', 'Profile picture', 'Followers',
       'Following', 'Listed', 'Tweet language (ISO 639-1)', 'Tweet Url',
       'Is a RT', 'Original Tweet User Name', 'User Mentions', 'Hashtags',
       'Symbols', 'Media', 'URLs'],
      dtype='object')

In [32]:
googl.head()

,Tweet Id,Date,Hour,User Name,Nickname,Bio,Tweet content,Favs,RTs,Latitude,...,Listed,Tweet language (ISO 639-1),Tweet Url,Is a RT,Original Tweet User Name,User Mentions,Hashtags,Symbols,Media,URLs
0,743031249682477057,2016-06-15,10:43,kimweeks,kimweeks77,NaN,RT @johnscharts: Similar to $GOOGL: $NFLX is s...,NaN,2.0,NaN,...,7.0,en,http://www.twitter.com/kimweeks77/status/74303...,True,@johnscharts,@johnscharts,NaN,"$GOOGL,$NFLX",http://pbs.twimg.com/media/Ck_Gw_nWgAISZ37.jpg,NaN
1,743031012406394880,2016-06-15,10:42,BullishChartPatterns,johnscharts,Chart pattern trader. Tweeting with friends an...,Similar to $GOOGL: $NFLX is showing reversal s...,3.0,2.0,37.54812,...,87.0,en,http://www.twitter.com/johnscharts/status/7430...,False,NaN,NaN,NaN,"$GOOGL,$NFLX",http://pbs.twimg.com/media/Ck_Gw_nWgAISZ37.jpg,NaN
2,743030242688696321,2016-06-15,10:39,Advertising News,AdNewsUS,The World of Advertising and Marketing Ideas,"#consumer, #jobs New #OT rules | ""OVERTIME: It...",NaN,NaN,NaN,...,93.0,en,http://www.twitter.com/AdNewsUS/status/7430302...,False,NaN,@YouTube,"#consumer,#jobs,#OT",$GOOGL,NaN,https://youtu.be/UFJaDm720FU
3,743029037858234368,2016-06-15,10:35,ProTradersNews,ProTradersNews,News for Pro Traders,$GOOGL:\n\nIs Dallas the Next Destination for ...,NaN,NaN,NaN,...,103.0,en,http://www.twitter.com/ProTradersNews/status/7...,False,NaN,NaN,NaN,$GOOGL,NaN,http://www.StockNewsWires.com/2016/06/15/googl...
4,743028676883779584,2016-06-15,10:33,dailystockplays.com,DailyStockPlays,"Live Day Trading Calls, Daily Swing Trading Li...",Unusual Call Activity https://t.co/4jDlRqXapi ...,NaN,NaN,37.25022,...,120.0,en,http://www.twitter.com/DailyStockPlays/status/...,False,NaN,NaN,NaN,"$KR,$WFT,$LOW,$PYPL,$MGM,$VXX,$GLD,$IYR,$LVS,$...",NaN,http://www.dailystockplays.com/Unusual-Call-Op...


In [43]:
googl=clean_data(googl)
googl

,Date,Tweet content
0,2016-06-15,RT @johnscharts: Similar to $GOOGL: $NFLX is s...
1,2016-06-15,Similar to $GOOGL: $NFLX is showing reversal s...
2,2016-06-15,"#consumer, #jobs New #OT rules | ""OVERTIME: It..."
3,2016-06-15,$GOOGL:\n\nIs Dallas the Next Destination for ...
4,2016-06-15,Unusual Call Activity https://t.co/4jDlRqXapi ...
5,2016-06-15,$SPY $FB $IWM $IBB $AMZN $GOOGL Pay Attention ...
6,2016-06-15,$MSFT $AAPL $GOOGL:\n\nApple Tries to Limit Go...
7,2016-06-15,Google Fiber explores Dallas https://t.co/lsdB...
8,2016-06-15,#YouTube is built on the back of stolen conten...
9,2016-06-15,"Andreas Halvorsen Buys Facebook, Amazon, Maste..."


In [44]:
googl = get_sentiment_score(googl)
print('Length: ',len(googl))
googl

Length:  14502


,Date,Tweet content,sentiment
5,2016-06-15,$SPY $FB $IWM $IBB $AMZN $GOOGL Pay Attention ...,-0.1027
8,2016-06-15,#YouTube is built on the back of stolen conten...,-0.4939
9,2016-06-15,"Andreas Halvorsen Buys Facebook, Amazon, Maste...",0.1779
10,2016-06-15,RT @MarketsRecap: These 3 Stocks Will Surprise...,0.2732
16,2016-06-15,These 3 Stocks Will Surprise Investors This Ea...,0.2732
21,2016-06-15,$GOOGL $FB French killer's Facebook broadcast ...,0.0772
22,2016-06-15,RT @DarbyOGill_: Sign up for free on 1broker w...,0.7351
24,2016-06-15,TLT Direxion Daily 20+ Year Treasury Bull 3x S...,0.4588
27,2016-06-15,RT @theflynews: On The Fly: Top stock stories ...,0.2023
28,2016-06-15,RT @sonycomeback: $SNE #SonyE3 BOOK VALUE 2X ...,-0.5514


In [45]:
googl = get_daily_sentiment(googl)
googl

,sentiment
Date,
2016-03-29,0.317342
2016-03-30,0.235119
2016-03-31,0.247812
2016-04-01,0.185321
2016-04-02,0.223353
2016-04-03,0.295681
2016-04-04,0.112185
2016-04-05,0.232705
2016-04-06,0.276270


In [46]:
googl = reset_index(googl)

In [47]:
googl

,sentiment,short,return,date
0,0.317342,0,0,2016-03-29
1,0.235119,0,0,2016-03-30
2,0.247812,0,0,2016-03-31
3,0.185321,0,0,2016-04-01
4,0.223353,0,0,2016-04-02
5,0.295681,0,0,2016-04-03
6,0.112185,0,0,2016-04-04
7,0.232705,0,0,2016-04-05
8,0.276270,0,0,2016-04-06
9,0.241570,0,0,2016-04-07


In [48]:
prices = get_stock_prices('GOOGL',googl)
prices

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-03-28,756.17,758.30,752.04,753.28,753.28,1083800
2016-03-29,753.68,767.18,748.29,765.89,765.89,2003100
2016-03-30,768.21,777.31,767.58,768.34,768.34,2019800
2016-03-31,768.34,769.08,758.25,762.90,762.90,1627700
2016-04-01,757.16,770.04,755.20,769.67,769.67,1583700
2016-04-04,769.51,772.44,761.79,765.12,765.12,1346200
2016-04-05,758.13,762.87,755.60,758.57,758.57,1223900
2016-04-06,757.84,768.42,756.30,768.07,768.07,1265600
2016-04-07,765.32,769.36,757.50,760.12,760.12,1274900


In [49]:
googl = match_open_close(googl,prices)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [50]:
googl

,sentiment,short,return,date,Open,Close
0,0.317342,0,0,2016-03-29,753.68,765.89
1,0.235119,0,0,2016-03-30,768.21,768.34
2,0.247812,0,0,2016-03-31,768.34,762.90
3,0.185321,0,0,2016-04-01,757.16,769.67
4,0.223353,0,0,2016-04-02,0.00,0.00
5,0.295681,0,0,2016-04-03,0.00,0.00
6,0.112185,0,0,2016-04-04,769.51,765.12
7,0.232705,0,0,2016-04-05,758.13,758.57
8,0.276270,0,0,2016-04-06,757.84,768.07
9,0.241570,0,0,2016-04-07,765.32,760.12


In [53]:
googl = calculate_return(short(googl))
googl

Total Return:  1061


,sentiment,short,return,date,Open,Close
0,0.317342,0,0,2016-03-29,753.68,765.89
1,0.235119,0,0,2016-03-30,768.21,768.34
2,0.247812,0,0,2016-03-31,768.34,762.90
3,0.185321,0,0,2016-04-01,757.16,769.67
4,0.223353,0,0,2016-04-02,0.00,0.00
5,0.295681,0,0,2016-04-03,0.00,0.00
6,0.112185,0,0,2016-04-04,769.51,765.12
7,0.232705,0,0,2016-04-05,758.13,758.57
8,0.276270,0,0,2016-04-06,757.84,768.07
9,0.241570,0,0,2016-04-07,765.32,760.12


In [52]:
googl

,sentiment,short,return,date,Open,Close
0,0.317342,0,0,2016-03-29,753.68,765.89
1,0.235119,0,0,2016-03-30,768.21,768.34
2,0.247812,0,0,2016-03-31,768.34,762.90
3,0.185321,0,0,2016-04-01,757.16,769.67
4,0.223353,0,0,2016-04-02,0.00,0.00
5,0.295681,0,0,2016-04-03,0.00,0.00
6,0.112185,0,0,2016-04-04,769.51,765.12
7,0.232705,0,0,2016-04-05,758.13,758.57
8,0.276270,0,0,2016-04-06,757.84,768.07
9,0.241570,0,0,2016-04-07,765.32,760.12


In [1]:
amzn_return

NameError: name 'amzn_return' is not defined

In [17]:
sbux_tweets = clean_data(pd.read_excel('sbux.xlsx','Stream'))
sbux_senti = get_sentiment_score(sbux_tweets)
sbux_senti = get_daily_sentiment(sbux_senti)
sbux_senti = reset_index(sbux_senti)
sbux = get_stock_prices('SBUX',sbux_senti)
sbux_senti = match_open_close(sbux_senti,sbux)
sbux_return = calculate_return(short(sbux_senti))

[*********************100%***********************]  1 of 1 downloaded


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Total Return:  21


In [14]:
cost_senti[cost_senti['sentiment'] < 0]

,sentiment,short,return,date,Open,Close
3,-0.034009,0,0,2016-03-13,0.00,0.00
43,-0.067000,0,0,2016-04-28,152.01,151.44
51,-0.016456,0,0,2016-05-06,147.06,146.44
57,-0.013379,0,0,2016-05-12,147.32,146.72
60,-0.230617,0,0,2016-05-15,0.00,0.00
85,-0.002406,0,0,2016-06-09,153.00,154.05


In [19]:
ea_tweets = clean_data(pd.read_excel('ea.xlsx','Stream'))
ea_senti = get_sentiment_score(ea_tweets)
ea_senti = get_daily_sentiment(ea_senti)
ea_senti = reset_index(ea_senti)
ea = get_stock_prices('EA',ea_senti)
ea_senti = match_open_close(ea_senti,trip)
ea_return = calculate_return(short(ea_senti))

[*********************100%***********************]  1 of 1 downloaded


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Total Return:  -210


In [20]:
ea_senti

,sentiment,short,return,date,Open,Close
0,0.458800,0,0,2016-03-27,0.00,0.00
1,0.101123,0,0,2016-03-28,62.61,62.51
2,0.196297,0,0,2016-03-29,62.79,65.58
3,0.296241,0,0,2016-03-30,65.93,66.58
4,0.318457,0,0,2016-03-31,66.00,66.50
5,0.530750,0,0,2016-04-01,66.05,67.23
6,-0.018880,0,0,2016-04-02,0.00,0.00
7,0.040173,0,0,2016-04-03,0.00,0.00
8,0.248050,0,0,2016-04-04,67.44,65.09
9,0.334782,0,0,2016-04-05,64.09,63.96
